In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import cv2
import pandas as pd

In [0]:
def trainImageReader(rootPath):
    image_data=[]
    os.chdir(rootPath)
    folders=os.listdir()
    for folder in folders:
        print(folder)
        os.chdir(os.path.join(rootPath,folder))
        files=os.listdir()
        for file in files:
            image_data.append([cv2.imread(os.path.join(rootPath,folder,file),cv2.IMREAD_GRAYSCALE),folder])
    return pd.DataFrame(image_data,columns=['Intensity','Class'])

In [4]:
image_df=trainImageReader('//content//gdrive//My Drive//Colab Notebooks//train')
#image_df=pd.read_csv('//content//gdrive//My Drive//Colab Notebooks//train//image_df.csv')

Common Chickweed
Maize
Scentless Mayweed
Loose Silky-bent
Common wheat
Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock


In [0]:
height=224
width=224

In [0]:
#Resize Images to have same size
image_resized=image_df['Intensity'].apply(lambda x:cv2.resize(x,(height,width),interpolation = cv2.INTER_AREA))

In [0]:
import numpy as np
image_intensity=np.empty((image_resized.shape[0],height,width,1))

In [0]:
#Reshape
image_resized_reshaped=image_resized.apply(lambda x:np.reshape(x,(height,width,1)))

In [9]:
image_resized_reshaped[0].shape

(224, 224, 1)

In [0]:
for i in range(image_df['Intensity'].shape[0]):
    image_intensity[i]=image_resized_reshaped[i]

In [0]:
#normalize
image_normalized=image_intensity/255

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le=LabelEncoder()
image_df['Class']=le.fit_transform(image_df['Class'])

In [0]:
y=pd.get_dummies(image_df['Class'],prefix='Class')

In [0]:
#test and validation split
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_test,y_train,y_test=train_test_split(image_normalized,np.array(y),test_size=0.05,random_state=1)

In [17]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Prepare the generator
datagen.fit(x_train)

Using TensorFlow backend.


In [0]:
#Building Model
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential

In [0]:
TRAIN = False
BATCH_SIZE = 32
EPOCHS = 10

In [20]:

    # Define Model
    model = Sequential()

    # 1st Conv Layer
    model.add(Convolution2D(64, (3, 3),padding='same', input_shape=(height, width,1)))
    model.add(Activation('relu'))
    
    # 2nd Conv Layer
    model.add(Convolution2D(64, (3, 3),padding='same', input_shape=(height, width,1)))
    model.add(Activation('relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # 3rd Conv Layer
    model.add(Convolution2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    
    # 4th Conv Layer
    model.add(Convolution2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    
    model.add(Dense(128))
    model.add(Activation('relu'))

    # Prediction Layer
    model.add(Dense(12))
    model.add(Activation('softmax'))

    # Loss and Optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Store Training Results
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
    callback_list = [early_stopping]

    # Train the model
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE), nb_epoch=EPOCHS, samples_per_epoch=1000,
              validation_data=(x_test, y_test), callbacks=callback_list)

W0701 20:09:53.179553 140392978139008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0701 20:09:53.226286 140392978139008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0701 20:09:53.234393 140392978139008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0701 20:09:53.275129 140392978139008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0701 20:09:53.361609 140392978139008 deprecation_wrapp

Epoch 1/10
31/31 [==============================] - 21s 675ms/step - loss: 2.6828 - acc: 0.1321 - val_loss: 2.4626 - val_acc: 0.1218
Epoch 2/10
31/31 [==============================] - 8s 265ms/step - loss: 2.4223 - acc: 0.1562 - val_loss: 2.4347 - val_acc: 0.1218
Epoch 3/10
31/31 [==============================] - 8s 267ms/step - loss: 2.4307 - acc: 0.1260 - val_loss: 2.4372 - val_acc: 0.1218
Epoch 4/10
31/31 [==============================] - 8s 267ms/step - loss: 2.4323 - acc: 0.1190 - val_loss: 2.4239 - val_acc: 0.1218
Epoch 00004: early stopping


In [21]:
train_eval=model.evaluate(x_train,y_train)

4512/4512 [==============================] - 11s 2ms/step


In [22]:
print("Accuracy on train data is",train_eval[1]*100)

Accuracy on train data is 13.85195035460993


In [23]:
valid_eval=model.evaluate(x_test,y_test)

238/238 [==============================] - 1s 3ms/step


In [24]:
print("Accuracy on validation data is",valid_eval[1]*100)

Accuracy on validation data is 12.184873987145785
